In [ ]:
import datatable as dt

In [ ]:
biogrid_mv = dt.fread("data/BIOGRID-MV-Physical-3.5.187.tab3.txt",  header=True,)

In [ ]:
with open(file="data/BIOGRID-MV-Physical-3.5.187.tab3.txt") as fi:
  ww = fi.readline().rstrip().split("\t")
biogrid_mv.names = ww



In [ ]:
biogrid_mv_human = biogrid_mv[ list(map(lambda x,y: (x==9606)&(y==9606), biogrid_mv["Organism Interactor A"].to_list()[0], biogrid_mv["Organism Interactor B"].to_list()[0])),:]

In [ ]:
edges = list(set(list(zip(biogrid_mv_human["Official Symbol Interactor A"].to_list()[0], biogrid_mv_human["Official Symbol Interactor B"].to_list()[0]))))

In [ ]:
edges = list(set(list(map(lambda x: tuple(sorted(x)), edges))))

In [ ]:
edges_zipped = list(zip(*edges))

In [ ]:
all_nodes = sorted(set.union(set(edges_zipped[0]), set(edges_zipped[1])))

In [ ]:
import torch
from torch_geometric.data import Data

In [ ]:
node2ind = dict(zip(all_nodes, range(len(all_nodes))))

In [ ]:
edges_zipped_indices = [list(map(lambda x: node2ind[x], edges_zipped[0])),
                        list(map(lambda x: node2ind[x], edges_zipped[0]))]


In [ ]:
edge_index = torch.tensor(edges_zipped_indices, dtype=torch.long)


In [1]:
from goatools.base import download_go_basic_obo
obo_fname = download_go_basic_obo()
from goatools.base import download_ncbi_associations
gene2go = download_ncbi_associations()
from goatools.obo_parser import GODag

obodag = GODag("go-basic.obo")



from __future__ import print_function
from goatools.associations import read_ncbi_gene2go

geneid2gos = read_ncbi_gene2go("gene2go", taxids=[9606])

requests.get(http://purl.obolibrary.org/obo/go/go-basic.obo, stream=True)
  WROTE: go-basic.obo

FTP RETR ftp.ncbi.nlm.nih.gov gene/DATA gene2go.gz -> gene2go.gz
  gunzip gene2go.gz
go-basic.obo: fmt(1.2) rel(2020-07-16) 47,259 GO Terms
DEPRECATED read_ncbi_gene2go: USE Gene2GoReader FROM goatools.anno.genetogo_reader
DEPRECATED read_ncbi_gene2go CALLED FROM: <ipython-input-1-dd0eaccd4c15> BY <module>
HMS:0:00:06.123295 333,299 annotations, 20,585 genes, 18,443 GOs, 1 taxids READ: gene2go 
18591 IDs in loaded association branch, BP


In [ ]:
from numpy import random
x = torch.tensor([random.randint(-1,2,size=len(all_nodes)),
     random.randint(-1,2,size=len(all_nodes)),
     random.randint(-1,2,size=len(all_nodes)),
     random.randint(-1,2,size=len(all_nodes))], dtype=torch.float).t()
y = torch.tensor(random.randint(0,4,len(all_nodes)), dtype=torch.long)
train_mask = torch.tensor(random.random_sample(size=len(all_nodes)) > .8, dtype=torch.bool)



  'GO:0050776',
  'GO:0050900',
  'GO:0070208',
  'GO:0071230',
  'GO:0085029'},
 1280: {'GO:0001501',
  'GO:0001502',
  'GO:0001894',
  'GO:0001958',
  'GO:0002062',
  'GO:0003007',
  'GO:0006029',
  'GO:0007417',
  'GO:0007601',
  'GO:0007605',
  'GO:0010468',
  'GO:0030198',
  'GO:0030199',
  'GO:0030903',
  'GO:0042472',
  'GO:0050776',
  'GO:0051216',
  'GO:0060021',
  'GO:0060174',
  'GO:0060272',
  'GO:0060351',
  'GO:0071599',
  'GO:0071773',
  'GO:0097065',
  'GO:2001240'},
 1281: {'GO:0001501',
  'GO:0007160',
  'GO:0007179',
  'GO:0007229',
  'GO:0007507',
  'GO:0009314',
  'GO:0009612',
  'GO:0018149',
  'GO:0021987',
  'GO:0030168',
  'GO:0030198',
  'GO:0030199',
  'GO:0034097',
  'GO:0035025',
  'GO:0042060',
  'GO:0043588',
  'GO:0048565',
  'GO:0050776',
  'GO:0050777',
  'GO:0060414',
  'GO:0071230',
  'GO:0097435',
  'GO:2001223'},
 1282: {'GO:0001569',
  'GO:0007420',
  'GO:0007528',
  'GO:0030198',
  'GO:0030855',
  'GO:0038063',
  'GO:0048514',
  'GO:0061304',
  

In [ ]:
mydata = Data(edge_index=edge_index, x = x, y = y, train_mask = train_mask, num_classes = 4)

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(mydata.num_node_features, 2)
        self.conv2 = GCNConv(2, mydata.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        import pdb
        # pdb.set_trace()

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
mydata = mydata.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(mydata)
    loss = F.nll_loss(out[mydata.train_mask], mydata.y[mydata.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
a, pred = model(mydata).max(dim=1)

In [ ]:
data = mydata
correct = int(pred[torch.logical_not(data.train_mask)].eq(data.y[torch.logical_not(data.train_mask)]).sum().item())

In [ ]:
acc = correct / int(torch.logical_not(data.train_mask).sum())
acc

In [ ]:
from torch_geometric.datasets import Planetoid

planetset = Planetoid(root='/tmp/Cora', name='Cora')

planets = planetset[0]

In [ ]:
planets.edge_index.dtype